# Mandatory Exercise - Session 5

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is to categorize the given pairs, print their most frequent WordNet synset, their corresponding least common subsumer (LCS) and their similarity using the following functions:

- Path Similarity
- Leacock-Chodorow Similarity
- Wu-Palmer Similarity
- Lin Similarity


In [1]:
# initial imports. Could also be done in the PC
import nltk
import numpy as np
import pandas as pd
import itertools

#additional set of imports
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic


#given set of pairs
pairs = [('the', 'DT'), ('man', 'NN'), ('swim', 'VB'), \
         ('with', 'PR'), ('a', 'DT'), ('girl', 'NN'), \
         ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), \
         ('whilst', 'PR'), ('the', 'DT'), ('woman', 'NN'), \
         ('walk', 'VB')]

d = {}
i = 1
perm = []
lcs = {}
definition = []

Now, for each pair, we will search for their most frequent WordNet sysnset. In the documentation we can find that there are also adjectives and adverbs listed as options but are not used in this given set of pairs. Listing it as a reference: https://wordnet.princeton.edu/documentation/wn1wn

In [2]:
for e in pairs:
    if e[0] not in d:
        if e[1] == 'NN':
            d[e[0]] = ['noun']
            d[e[0]] += [wn.synset(e[0]+'.n.01')]
        elif e[1] == 'VB':
            d[e[0]] = ['verb']
            d[e[0]] += [wn.synset(e[0]+'.v.01')]
            
#verification that it is properly stored
for keys,values in d.items():
    print(keys, values)
            

man ['noun', Synset('man.n.01')]
swim ['verb', Synset('swim.v.01')]
girl ['noun', Synset('girl.n.01')]
boy ['noun', Synset('male_child.n.01')]
woman ['noun', Synset('woman.n.01')]
walk ['verb', Synset('walk.v.01')]


For each of the pairs that was found to be on WordNet, we now get their corresponding least commmon subsumer. That is the most specific common ancestor (hypernym) of two concepts found in a given ontology. For example, the LCS of moose and kangaroo in WordNet is mammal.

In [3]:
print(d['man'], d['swim'])
d['man'].lowest_common_hypernyms(d['swim'])
synset('man.n.01').lowest_common_hypernyms(synset('apple.n.01'))

['noun', Synset('man.n.01')] ['verb', Synset('swim.v.01')]


AttributeError: 'list' object has no attribute 'lowest_common_hypernyms'

In [ ]:
for key in perm:
    for alt in perm[1:]: 
        lcs[i] = [key, alt, d[key].lowest_common_hypernyms(d[alt])]
        i += 1
    perm = perm[1:]
#verification that it is properly stored
for keys,values in lcs.items():
    print(keys, values)

We then plot the results, not only to better visualize the specific x and y coordinates that we have obtained and store but also to connect the dots and evaluate the overall increase in performance and benchmarking against the other models. 

In [ ]:
#adding the necessary packages for plotting
import matplotlib.pyplot as plt

#preparing and plotting the accuracy graph
x = train_stop
plt.figure()
plt.plot(x, accuracy['HMM'], label='HMM')
plt.plot(x, accuracy['TnT'], label='TnT')
plt.plot(x, accuracy['PER'], label='PER')
plt.plot(x, accuracy['CRF'], label='CRF')

#adding the legend showing the plot
plt.xlabel('Number of Sentences')
plt.ylabel('Model Accuracy')
plt.title('Part Of Speech Models')
plt.legend()
plt.show()

<b>Which model would you select?</b> Justify the answer.

Based on the initial graph, both CRF and PER seem like the better performing algorithms. If we could only make a decision based on this data, we would pick the Perceptron model.

In order to pick the best overall model, another relevant measure is the speed of the system. Because of this, we also have implemented a timer in between each of the models and will plot it accordingly to figure out which algorithms were more effective. 

In [ ]:
#preparing and plotting the accuracy graph
x = train_stop
plt.figure()
plt.plot(x, time['HMM'], label='HMM')
plt.plot(x, time['TnT'], label='TnT')
plt.plot(x, time['PER'], label='PER')
plt.plot(x, time['CRF'], label='CRF')

#adding the legend showing the plot
plt.xlabel('Number of Sentences')
plt.ylabel('Model Execution Time')
plt.title('Part Of Speech Models')
plt.legend()
plt.show()

In this case, PER seems to be the most effective -accuracy wise- and second most time efficient algorithm. After having performed this additional execution time analysis <b> we are reinsured in our conclusion to pick the Perceptron Model.</b> 

## Conclusions
Over the development of this lab, we have implemented 4 different POS Models, tested them with different segments of the treebank corpus and trained them with another set of segments. From those results, we have plotted their performanced based on accuracy and execution time and have selected the best performing model for this specific case, which happened to be the Perceptron Model. 

Another thing to highlight is that the HiddenMarkovModel looks really efficient timewise. It is likely that its just not in the desired set of conditions regarding the amount of data to perform correctly. 
